In [ ]:
import torch
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
torch.cuda.device_count(), device

(0, device(type='cpu'))

In [ ]:
import os
import sys
IS_GCOLAB = 'google.colab' in sys.modules

In [37]:
from pathlib import Path

_base_libpath = Path(
    "/content/drive/MyDrive" if IS_GCOLAB else f"{os.environ['HOME']}"
) / "_self_research/24.09.world_cuisine"

assert _base_libpath.exists()

os.chdir(_base_libpath)

In [ ]:
if IS_GCOLAB:
    raise NotImplementedError()
    # from google.colab import drive
    # drive.mount('/content/drive')
    # # os.environ.setdefault("", "1")
    # # os.system("pip install transformers==4.33.1")
    # os.system("pip install transformers sudachitra accelerate")
    # os.system("pip install langchain tiktoken openai")
    # os.system("pip install -U langchain-community")
    # os.system("pip install -U datasets pyarrow")

else:
    # raise NotImplementedError()
    print(">>> Not on Google Colab <<<")

>>> Not on Google Colab <<<


---

In [ ]:
# import datasets
# from datasets import load_dataset
# from torch.utils.data import DataLoader
# from sentence_transformers import SentenceTransformer

import os
import requests
import pickle
from PIL import Image
from glob import glob
from pathlib import Path
import numpy as np
import pandas as pd

import time
from tqdm import tqdm

import re
import json
from collections import Counter

In [ ]:
df = pd.read_csv("resources/WorldCuisines - Food.2410080637.tsv", sep="\t")
for (i, r) in df.iterrows():
    if not r.loc["Image1 name / URL"]:
        for k in range(2,9):
            assert not r.loc[f"Image{k} name / URL"]

In [ ]:
# df = pd.read_csv("resources/WorldCuisines - Food.2410032035.tsv", sep="\t")
df = pd.read_csv("resources/WorldCuisines - Food.2410080637.tsv", sep="\t")
print(f"Cuisine counts: {len(df):6,}")
df = df.loc[df.loc[:,'Image1 name / URL'].notna()].reset_index(drop=True)
print(f"  w/ imgs only: {len(df):6,}")

Cuisine counts:  2,414
  w/ imgs only:  2,240


In [ ]:
fps = []
for _ in range(1,9):
    wew = df.loc[:,f"Image{_} name / URL"].dropna()
    fps.extend(wew.tolist())
fps = pd.Series(fps)

In [ ]:
wew = fps.apply(lambda t: re.sub(r"(\?|\?\d{14}=?|\?&?download){0,2}$", '', t.lower()))

for i, x in enumerate(wew):
    if not (x.endswith(".jpg") | x.endswith(".jpeg") | x.endswith("webp")
    | x.endswith(".png") | x.endswith(".gif") | x.endswith(".tif")):
        print(i, x)

assert not wew[~(
    wew.str.endswith(".jpg") | wew.str.endswith(".jpeg") | wew.str.endswith("webp")
    | wew.str.endswith(".png") | wew.str.endswith(".gif") | wew.str.endswith(".tif")
)].tolist()
# print(len(wew), len(wew.unique()))

In [ ]:
print(f"Image Count: {len(fps):,}")
fps = fps[~wew.duplicated()].reset_index(drop=True)
print(f"  Unique # : {len(fps):,}")

Image Count: 6,103
  Unique # : 6,058


---
License Crawler

In [ ]:
url = get_wikimedia_image_license("https://upload.wikimedia.org/wikipedia/commons/2/2b/Kalam_dolmasi_Azerbaijan.jpg?download")

In [ ]:
from bs4 import BeautifulSoup
import csv
import os
from tqdm import tqdm

#di-cache biar gak extract web yg udah pernah ke-cache sebelomnya
cache_data_dict = {}
license_cache_fp = "resources/license_cache_data.csv"

if os.path.exists(license_cache_fp):
    # print(f"loading existing license cache ({license_cache_fp}) ..")
    with open(license_cache_fp, 'r') as f:
        reader = csv.reader(f)
        for row in reader:
            cache_data_dict[row[0]] = row[1]
    print(f"loaded existing license cache. ({license_cache_fp})")


def extract_website(url):
    if pd.isna(url):
        return None
    else:
        x = re.search("https?://(.+?)/", url)
        if not x:
            return None
        else:
            return x.group(1)

# convert to common wikimedia url to extract url
def convert_media_url_to_commons_page(media_url):
    # Extract the file name
    parts = media_url.split("/")
    file_name_1 = parts[-1]
    file_name_2 = parts[-2]  # The last part of the URL is the file name
    # check if [number]px in the file nbame then extract file_name_2
    if re.match(r"\d+px", file_name_1):
        file_name = file_name_2
    else:
        file_name = file_name_1

    # Construct the Wikimedia Commons page URL
    commons_page_url = f"https://commons.wikimedia.org/wiki/File:{file_name}"
    return commons_page_url

# license checker:
def get_wikimedia_image_license(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")
    license_shorts = list(map(lambda l: l.text.strip(), soup.find_all("span", {"class": "licensetpl_short"})))
    license_shorts_flt = list(filter(lambda l: l.startswith("CC"), license_shorts))
    license_short = license_shorts_flt[0] if len(license_shorts_flt) > 0 else license_shorts[-1] if len(license_shorts) > 0 else None
    if license_short:
        return license_short

invalid_url = 0
invalid_license = 0

extracted_license = []
loaded_from_cache = 0
new_cache = 0

for url in tqdm(fps):
    base_url = extract_website(url)
    if not base_url or base_url != "upload.wikimedia.org":
        invalid_url += 1
        extracted_license.append((url, None))
        continue

    convert_url = convert_media_url_to_commons_page(url)

    if url in cache_data_dict:
        license = cache_data_dict[url]
        loaded_from_cache += 1
    else:
        license = get_wikimedia_image_license(convert_url)
        if license != None:
            cache_data_dict[url] = license
            new_cache += 1
        else:
            print("Found invalid license: ", url)

        with open(license_cache_fp, 'a') as f:
            writer = csv.writer(f)
            writer.writerow([url, license])
            # for key, value in cache_data_dict.items():
            #     writer.writerow([key, value])

    extracted_license.append((url, license))

print(f"\nLoaded from Cache: {loaded_from_cache:6,}")
print(f"\tNew Cache: {new_cache:6,}  ({len(fps)-loaded_from_cache} skipped)")

# for e in extracted_license:
#     print(e)


loaded existing license cache. (resources/license_cache_data.csv)


  5%|███████▋                                                                                                                                                               | 279/6088 [00:00<00:15, 365.68it/s]

Found invalid license:  https://upload.wikimedia.org/wikipedia/en/8/8b/Real_lo_mein.jpg?download


 16%|██████████████████████████▊                                                                                                                                            | 977/6088 [00:01<00:06, 780.47it/s]

Found invalid license:  https://upload.wikimedia.org/wikipedia/commons/thumb/0/01/Amish_shoo-fly_pies%2C_made_from_a_gooey_molasses-based_filling_and_sometimes_topped_with_chocolate_cream._Lancaster%2C_Pennsylvania_LCCN2011634806.tif/lossy-page1-1348px-Amish_shoo-fly_pies%2C_made_from_a_gooey_molasses-based_filling_and_sometimes_topped_with_chocolate_cream._Lancaster%2C_Pennsylvania_LCCN2011634806.tif.jpg?download


...

Found invalid license:  https://upload.wikimedia.org/wikipedia/commons/thumb/5/5e/SNR-Bite1_Final.tif/lossy-page1-800px-SNR-Bite1_Final.tif.jpg?download


 66%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████▊                                                        | 4026/6088 [00:07<00:02, 699.88it/s]

Found invalid license:  https://upload.wikimedia.org/wikipedia/commons/thumb/6/69/Pappadum.tif/lossless-page1-1614px-Pappadum.tif.png?download


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 6088/6088 [00:08<00:00, 728.73it/s]

Found invalid license:  https://upload.wikimedia.org/wikipedia/en/c/cd/RiceCrepe.jpg?download

Loaded from Cache:  6,075
	New Cache:      0  (13 skipped)


---
Check Image size

In [ ]:
# assert all(fps[fps.str.startswith("http")].str.startswith("https://"))

In [ ]:
def get_img_size(fp):
    headers = {'User-Agent': 'CoolBot/0.0 (https://example.org/coolbot/; coolbot@example.org)'}
    try:
        if fp.startswith("https://"):
            response = requests.get(fp, headers=headers, stream=True, timeout=8)
            response.raise_for_status()
            _img = response.raw
        else:
            _img = _base_libpath / "images" / fp
        image = Image.open(_img).convert('RGB')
        return image.size
    except Exception as e:
        # print(fp, e)
        return None

In [ ]:
from multiprocessing import Pool

In [ ]:
%%time
with Pool(processes=32) as pool:
    async_result = pool.map_async(get_img_size, fps.tolist(), 16)
    async_result.wait()

In [ ]:
sizes = async_result.get()

In [ ]:
len(sizes), len(list(filter(None, sizes))), f"{len(list(filter(None, sizes)))/len(sizes):.1%}"

In [ ]:
# Previous count when total images was 1871

plot image size distribution

In [ ]:
import seaborn as sns

In [ ]:
df = pd.DataFrame(list(filter(None, sizes)), columns=["width", "height"])

In [ ]:
df

In [ ]:
df["hue"] = ""

In [ ]:
g = sns.JointGrid(df, x="width", y="height", hue="hue")
g.plot_joint(sns.scatterplot, s=20, alpha=.1)
g.plot_marginals(sns.histplot, kde=True, linewidth=0, palette=["g"])

as of 240919

In [ ]:
g = sns.JointGrid(df.loc[df.width.le(3072)&df.height.le(2048)], x="width", y="height", hue="hue")
g.plot_joint(sns.scatterplot, s=30, alpha=.2)
g.plot_marginals(sns.histplot, kde=True, linewidth=.2, palette=["g"])

---
---
---

In [ ]:
df_all = pd.read_csv("resources/WorldCuisines - Food.2410032035.tsv", sep="\t")
print(f"Cuisine counts: {len(df_all):6,}")
df_w_img = df_all.loc[df_all.loc[:,'Image1 name / URL'].notna()].reset_index(drop=True)
print(f"   has images : {len(df_w_img):6,}")
df_w_reg = df_all[df_all.loc[:,'Region1'].notna()].reset_index(drop=True)
print(f"   has Region1: {len(df_w_reg):6,}")

Cuisine counts:  2,580
   has images :  2,240
   has Region1:  2,416


In [ ]:
print("--- No Images stats ---")
# print(f"Has Region1, w/o imgs. #: {sum(df_all.loc[:,'Region1'].notna())-len(df_w_img):6,}")
print(f"w/ Region1, w/o imgs.: {len(df_w_reg[df_w_reg.loc[:,'Image1 name / URL'].isna()]):6,}")
print(f"    =>  w/  Text Desc. # : {len(df_w_reg[df_w_reg.loc[:,'Text Description'].notna() & df_w_reg.loc[:,'Image1 name / URL'].isna()]):6,}")
print(f"    =>  w/o Text Desc. # : {len(df_w_reg[df_w_reg.loc[:,'Text Description'].isna()]):6,}")

--- No Images stats ---
w/ Region1, w/o imgs.:    177
    =>  w/  Text Desc. # :    174
    =>  w/o Text Desc. # :      4


In [ ]:
# print("NaN categories also got.\nNo Cuisine also got..\n:\\")
df_w_reg[df_w_reg.loc[:,"Coarse-grained categories"].isna()|df_w_reg.loc[:,"Cuisines"].isna()].iloc[:,:-17]

,Count,Name,Assigned,Alias,Wikipedia Link,Coarse-grained categories,Fine-grained categories,Cuisines,Associated Cuisines,Area,Countries,Region1,Region2,Region3,Region4,Region5,Text Description
37,1,Albaloo polo,NaN,"[{'آلبالو پلو': 'Persian'}, {'albaloo polow ba...",https://en.wikipedia.org/wiki/Albaloo_polo,NaN,NaN,"[""Iranian""]","[""Shiraz""]","[""Shiraz""]","[""Iran""]",South Asia,NaN,NaN,NaN,NaN,The dish is made with rice and sour cherries u...
1664,1,Lancashire hotpot,Hanyang Zhao,[{'option': 'no_alias'}],https://en.wikipedia.org/wiki/Lancashire_hotpot,NaN,NaN,"[""British""]","[""British""]","[""England""]","[""England""]",Western Europe,NaN,NaN,NaN,NaN,This dish consists of lamb or mutton and onion...


In [ ]:
print("Got categories and cuisines, but no description..")
df_w_reg[df_w_reg.loc[:,"Coarse-grained categories"].notna()&df_w_reg.loc[:,"Cuisines"].notna()&df_w_reg.loc[:,"Text Description"].isna()].iloc[:,:-17]

Got categories and cuisines, but no description..


,Count,Name,Assigned,Alias,Wikipedia Link,Coarse-grained categories,Fine-grained categories,Cuisines,Associated Cuisines,Area,Countries,Region1,Region2,Region3,Region4,Region5,Text Description
417,1,French Fries,NaN,"[{'chips': 'not_stated'}, {'finger chips': 'In...",https://en.wikipedia.org/wiki/French_fries,"[""snack"",""potato""]","[""deep-fried potato""]","[""French"", ""Belgian""]","[""French"", ""Belgian""]","[""France"", ""Belgium""]","[""France"", ""Belgium""]",Western Europe,NaN,NaN,NaN,NaN,NaN
636,1,Butter pecan,NaN,"[{""name"":""John Doe"", ""age"":30, ""city"":""New Yor...",https://en.wikipedia.org/wiki/Butter_pecan,"[""dessert""]","[""ice cream""]","[""American""]","[""American""]","[""America""]","[""United States""]",North America,NaN,NaN,NaN,NaN,NaN
2226,1,Pickled pigs' feet,Jodie,[{'no_alias': 'not_stated'}],https://en.wikipedia.org/wiki/Pickled_pigs%27_...,"[""meat""]","[""pork""]","[""Mexican"", ""Chinese"", ""French"", ""Scandinavian...","[""Mexican"", ""Chinese"", ""French"", ""Scandinavian...","[""Mexico"", ""China"", ""France"", ""Scandinavia"", ""...","[""Mexico"", ""China"", ""France"", ""Germany""]",North America,East Asia,Western Europe,NaN,NaN,NaN
2362,1,Caravane cheese,Fariz,[{'no_alias': []}],https://en.wikipedia.org/wiki/Caravane_cheese,"[""cheese""]","[""camel milk"", ""cheese""]","[""Mauritanian""]","[""Mauritanian""]","[""Mauritania""]","[""Mauritania"", ""Senegal""]",North Africa,NaN,NaN,NaN,NaN,NaN


Let's check the stats

In [ ]:
df = df_w_reg[df_w_reg.loc[:,"Coarse-grained categories"].notna()&df_w_reg.loc[:,"Cuisines"].notna()&df_w_reg.loc[:,"Text Description"].notna()]
print(f"Somewhat OK count: {len(df)}")

Somewhat OK count: 2410


In [ ]:
# @title
def parse_txt(t):
    if t is np.nan:
        return []
    t = t.replace('"Liberia"m"Cote D\'Ivoire"', '"Liberia", "Cote D\'Ivoire"')
    t = t.replace("''", '"')
    t = re.sub(r"[‘'’“]", '"', t)
    t = re.sub(r'", ([A-Z])', r'", "\1', t)
    t = re.sub(r'([a-z])\]', r'\1"]', t)
    t = re.sub(r'([a-z]), "', r'\1", "', t)
    t = (t
            .replace('" "', '", "')
            .replace('"."', '", "')
            .replace(", and ", ", ")
            .replace('d"Ivoire', "d'Ivoire")
            .replace('D"Ivoire', "D'Ivoire")
        )
    try:
        t = json.loads(t)
    except Exception as e:
        print(" >>> lho:", t)
        raise e
    return t


In [ ]:
entries = df.loc[:,[
    'Coarse-grained categories', 'Fine-grained categories', 'Cuisines',
    'Associated Cuisines', 'Area', 'Countries'
    ]]

In [ ]:
counters = {}
for c in range(6):
    counter = Counter([e.lower() for _ in entries.iloc[:,c].apply(parse_txt) for e in _])
    counters[entries.columns[c]] = counter
    print(entries.columns[c], f"  (total:  {sum(counter.values()):,})")
    print(counter, end="\n\n")

In [ ]:
print("List of countries and cuisines so far:")
print(*sorted(counters["Cuisines"].keys() | counters["Associated Cuisines"].keys() | counters["Countries"].keys()), sep="\n")

In [ ]:
for c, cnt in sorted((counters["Cuisines"] + counters["Countries"] + counters["Associated Cuisines"]).items()):
    print(c, cnt, sep='\t')

---
---
---

Translations Check for Location & Cuisines

In [ ]:
df_all = pd.read_csv("resources/WorldCuisines - Food.2410032035.tsv", sep="\t")
df_w_reg = df_all[df_all.loc[:,'Region1'].notna()].reset_index(drop=True)
df = df_w_reg[df_w_reg.loc[:,"Coarse-grained categories"].notna()&df_w_reg.loc[:,"Cuisines"].notna()&df_w_reg.loc[:,"Text Description"].notna()]

In [ ]:
lnc = pd.read_csv("resources/WorldCuisines - Location&Cuisine.2410032055.tsv", sep="\t", header=[0, 1]).iloc[:,1:]

In [ ]:
lnc_en_loc = lnc["English", "<LOCATION>"]
lnc_en_cui = lnc["English", "<CUISINE>"]

In [ ]:
countries = pd.Series(sorted(set(c for c_str in df.Countries for c in json.loads(c_str))))
countries_w_keys = countries.loc[countries.isin(lnc_en_loc)].tolist()
countries_wo_keys = countries.loc[~countries.isin(lnc_en_loc)].tolist()

In [ ]:
cuisines = pd.Series(sorted(set(c for c_str in df.Cuisines for c in json.loads(c_str))))
cuisines_w_keys = cuisines.loc[cuisines.isin(lnc_en_cui)].tolist()
cuisines_wo_keys = cuisines.loc[~cuisines.isin(lnc_en_cui)].tolist()

In [ ]:
cuisines_asc = pd.Series(sorted(set(c for c_str in df["Associated Cuisines"] for c in json.loads(c_str))))
cuisines_asc_w_keys = cuisines_asc.loc[cuisines_asc.isin(lnc_en_cui)].tolist()
cuisines_asc_wo_keys = cuisines_asc.loc[~cuisines_asc.isin(lnc_en_cui)].tolist()

In [ ]:
print(", ".join(cuisines[~cuisines.isin(cuisines_asc)].map(lambda t: f'"{t}"')))

"Andalusian", "Andean", "Andhra", "Cagayan", "Centrafrican", "Chaoshan", "Chinese Indonesian", "Chinese Peruvian", "East European", "Istrian", "Mesopotamian", "New Orleans", "Taishan", "Yangzhou", "Yemenite Jewish", "Yucatecan"


In [ ]:
print("\n".join(countries_all_wo_keys))

In [ ]:
cuisines_all_wo_keys = sorted(set(cuisines_wo_keys + cuisines_asc_wo_keys))
print("\n".join(cuisines_all_wo_keys))

In [ ]:
lnc["idx_loc"] = lnc_en_loc
lnc["idx_cui"] = lnc_en_cui

In [ ]:
base_keys_w_data = sorted(set(
    lnc.set_index("idx_loc", drop=True).loc[countries_w_keys]["Base key", "Base key"].tolist() +
    lnc.set_index("idx_cui", drop=True).loc[sorted(set(cuisines_w_keys+cuisines_asc_w_keys))]["Base key", "Base key"].tolist()
))
len(base_keys_w_data)

307

In [ ]:
lnc.drop(("idx_cui", ""), axis=1).set_index("idx_loc", drop=True).loc[sorted(countries_w_keys)].to_csv("resources/WorldCuisines - L&C.countries.tsv", sep='\t')
lnc.drop(("idx_loc", ""), axis=1).set_index("idx_cui", drop=True).loc[sorted(set(cuisines_w_keys+cuisines_asc_w_keys))].to_csv("resources/WorldCuisines - L&C.cuisines.tsv", sep='\t')
lnc.loc[~lnc["Base key", "Base key"].isin(base_keys_w_data)].drop([("idx_loc",""), ("idx_cui","")], axis=1).to_csv("resources/WorldCuisines - L&C.notfound.tsv", sep='\t')

---
---
---

In [ ]:
raise NotImplementedError("Beyond this point is just scratch pad.")

In [ ]:
!pip install openpyxl

  Using cached openpyxl-3.1.5-py2.py3-none-any.whl.metadata (2.5 kB)
  Using cached et_xmlfile-1.1.0-py3-none-any.whl.metadata (1.8 kB)
Using cached openpyxl-3.1.5-py2.py3-none-any.whl (250 kB)
Using cached et_xmlfile-1.1.0-py3-none-any.whl (4.7 kB)


In [ ]:
# https://github.com/gentaiscool/world_cuisines/blob/features/aliases_crawler/src/aliases_scraper/outputs.xlsx
df = pd.read_excel("resources/food_name_multiling.xlsx")

In [ ]:
df = df.reset_index(drop=True).loc[:,["Name", "Wikipedia Link", "json_aliases"]]

In [ ]:
def got_en_in_json():
    for j in df.json_aliases:
        if "en - English" in json.loads(j).keys():
            return True
    return False
assert not got_en_in_json()

In [ ]:
df.to_csv("resources/food_name_multiling.csv")